<a href="https://colab.research.google.com/github/chanapacha/my-pantip-scraper/blob/main/Creating_the_PantipScraper_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrape data


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

scrape จากเว็บ pantip ใน tag ประกันชีวิต

In [2]:
r = requests.get('https://pantip.com/tag/ประกันชีวิต')
s = BeautifulSoup(r.content, 'html.parser')

In [3]:
r

<Response [200]>

In [4]:
s.find('a', {'class': 'gtm-latest-topic gtm-topic-layout-compact gtm-topic-type-filter-all'})

<a class="gtm-latest-topic gtm-topic-layout-compact gtm-topic-type-filter-all" href="https://pantip.com/topic/42660235" target="_blank"><i class="pantip-icons md-18 pt-ic-topic_type_talking"></i>ก้าวแรกเป็นตัวแทนประกัน กว่าจะได้ลูกค้ารายแรกเหนื่อยไหมคะ</a>

In [5]:
s.find('a', {'class': 'gtm-latest-topic gtm-topic-layout-compact gtm-topic-type-filter-all'})['href']

'https://pantip.com/topic/42660235'

In [3]:
links = []

for topic in s.find_all('li', {'class': 'pt-list-item'}):
  link = topic.find('a')['href']
  links.append(link)

In [7]:
links[:5]

['https://pantip.com/topic/42660235',
 'https://pantip.com/topic/42660160',
 'https://pantip.com/topic/42659926',
 'https://pantip.com/topic/42655839',
 'https://pantip.com/topic/42654344']

scrape หัวข้อกระทู้

In [4]:
request = requests.get('https://pantip.com/topic/42660235')
beauty = BeautifulSoup(request.content, 'html.parser')

In [5]:
request

<Response [200]>

In [6]:
beauty.find('h2', {'class': 'display-post-title'})

<h2 class="display-post-title">ก้าวแรกเป็นตัวแทนประกัน กว่าจะได้ลูกค้ารายแรกเหนื่อยไหมคะ</h2>

In [7]:
beauty.find('h2', {'class': 'display-post-title'}).text

'ก้าวแรกเป็นตัวแทนประกัน กว่าจะได้ลูกค้ารายแรกเหนื่อยไหมคะ'

รายละเอียดกระทู้

In [8]:
beauty.find('div', {'class': 'display-post-story'})

<div class="display-post-story">
								กำลังเริ่มเข้าสู่ตัวแทนขายประกันค่ะ อยากขอเทคนิค ความรู้ หรือแชร์ประสบการณ์การปิดเคสแรก<br/>
เรากำลังเร่งศึกษาแผนแต่ละตัวให้เกิดความมั่นใจ แล้วเสนอคนใกล้ๆตัว<br/>
จริงๆเกริ่นๆไปบ้าง แต่ก็ยังปิดเคสไม่ได้เลยค่ะ กลัวว่าวันหนึ่งตัวเองจะท้อค่ะ																															</div>

In [9]:
beauty.find('div', {'class': 'display-post-story'}).text.strip()

'กำลังเริ่มเข้าสู่ตัวแทนขายประกันค่ะ อยากขอเทคนิค ความรู้ หรือแชร์ประสบการณ์การปิดเคสแรก\nเรากำลังเร่งศึกษาแผนแต่ละตัวให้เกิดความมั่นใจ แล้วเสนอคนใกล้ๆตัว\nจริงๆเกริ่นๆไปบ้าง แต่ก็ยังปิดเคสไม่ได้เลยค่ะ กลัวว่าวันหนึ่งตัวเองจะท้อค่ะ'

# scrape จาก tag ที่เกี่ยวข้องกับ "ประกัน"

In [11]:
tag_list = ['ประกันภัยการเดินทาง', 'ประกันวินาศภัย', 'ประกันชีวิต', 'ประกันออนไลน์', 'ประกันสุขภาพ', 'ประกันสินเชื่อบ้าน']
title = []
story = []
source = []
url = []

In [12]:
url_pre = 'https://pantip.com/tag/'

for tag in tag_list:
  re = requests.get(url_pre + tag)
  soup = BeautifulSoup(re.content, 'html.parser')

# Extract Topic URLs
  for topic in soup.find_all('li', {'class': 'pt-list-item'}):
    link = topic.find('a')['href']
    if link not in url:
      url.append(link)

for u in url[0:5]: # ในที่นี้จะกำหนดให้ใช้แค่ 5 url เพื่อความรวดเร็วในการแสดงตัวอย่าง
  req = requests.get(u)
  bs = BeautifulSoup(req.content, 'html.parser')

# Extracts the topic title
  for k in bs.find_all('div', {'class': 'display-post-status-leftside'}):
    topic_element = k.find('h2', {'class': 'display-post-title'})

    if topic_element is not None:
        topic = topic_element.text.strip()
        topic = topic.replace('\n', '')
        topic = topic.replace('\t', '')
        topic = topic.replace('\xa0', '')
        topic = topic.replace('*', '')
        topic = topic.replace('\u200b', '')
        title.append(topic)

# Extracts the story of each title
  for j in bs.find_all('div', {'class': 'display-post-story-wrapper'}):
    content = j.find('div', {'class': 'display-post-story'}).text.strip()
    content = content.replace('\n', '')
    content = content.replace('\t', '')
    content = content.replace('\xa0', '')
    content = content.replace('*', '')
    content = content.replace('\u200b', '')
    story.append(content)

## dataframe

In [22]:
data = pd.DataFrame([title, story, source, url[0:5]]).transpose()
data.columns = ['title', 'story', 'source', 'url']
data['source'] = 'pantip'
data

,title,story,source,url
0,ใคคพอรู้บ้างว่า ประกันเดินทางตอนไปต่างประเทศขอ...,พอดีกำลังจะไปเวียดนาม เลยไม่มั่นใจว่า เวลาจะ a...,pantip,https://pantip.com/topic/42658999
1,DTAC Go Travel ฟรีประกันการเดินทาง ต้องทำอย่าง...,สมัครแพคเกจ Go Travel 549 บาทไว้ของ DTAC เห็นม...,pantip,https://pantip.com/topic/42639023
2,ช่วยแนะนำประกันภัยรถยนต์ตั้งแต่ 3 + ขึ้นไปค่ะ,Honda civic ปี 2003ขอ 3 + ขึ้น ไป น่า จะ ดี กว...,pantip,https://pantip.com/topic/42635808
3,"Lounge Key, Dragon Pass, Priority Pass ต่างกัน...",สงสัยเรื่อง Lounge สนามบินทั้งสามครับ Lounge K...,pantip,https://pantip.com/topic/42626591
4,ต่อประกันรถยนต์ไม่ได้ Rabbit Care ไม่ยอมยกเลิก...,จ่ายเงินไปแล้ว แต่ออกกรมธรรม์ไม่ได้ แจ้งทาง Ra...,pantip,https://pantip.com/topic/42623033


# Apply Using Selenium to get more URL link

In [24]:
# install selenium
!pip install selenium
# install chrom driver
!apt-get update
!apt-get install chromium chromium-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,251 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,748 kB]
Hit:10 https://ppa.launchpadco

In [25]:
# import selenium library
from selenium import webdriver

# import beatifulsoup and other necessary lib for web scraping
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [29]:
# Selenium settings
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [30]:
tag_list = ['ประกันภัยการเดินทาง', 'ประกันวินาศภัย', 'ประกันชีวิต', 'ประกันออนไลน์', 'ประกันสุขภาพ', 'ประกันสินเชื่อบ้าน']
text = []
source = []
url = []
url_pre = 'https://pantip.com/tag/'

# Initialize the Chrome driver
driver = web_driver()

# Set the number of times to scroll
scroll_iterations = 2  # Adjust this number as needed

for tag in tag_list:
    url_tag = url_pre + tag
    driver.get(url_tag)

    for _ in range(scroll_iterations):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

    # Now that the page has scrolled the desired number of times, parse the HTML with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for topic in soup.find_all('li', {'class': 'pt-list-item'}):
        link = topic.find('a')['href']
        if link not in url:
            url.append(link)

# Close the Selenium webdriver
driver.quit()

In [31]:
# ได้ URL link มาเยอะกว่าเดิม
len(url)

834

# Creating the PantipScraper Class
create by Chanapa Chareesan

In [27]:
%%writefile PantipTagScraper.py
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

class PantipScraper:
    def __init__(self, scroll_iterations=0):
        self.scroll_iterations = scroll_iterations

    # Initialize the Chrome webdriver with specified options
    def web_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--verbose")
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument("--window-size=1920, 1200")
        options.add_argument('--disable-dev-shm-usage')
        return webdriver.Chrome(options=options)

    # Function to get URLs of topics based on tag list
    def get_topic_urls(self, tag_list):
        url_pre = 'https://pantip.com/tag/'
        urls = []

        # Initialize the Chrome driver
        driver = self.web_driver()
        try:
            for tag in tag_list:
                url_tag = url_pre + tag
                driver.get(url_tag)

                # Scroll the page down to load more topics if specified
                for _ in range(self.scroll_iterations):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)  # Wait for the page to load after scrolling

                # Parse the HTML content with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Find all topic links and append them to the list of URLs
                for topic in soup.find_all('li', {'class': 'pt-list-item'}):
                    link = topic.find('a')['href']
                    if link not in urls:
                        urls.append(link)
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            driver.quit()  # Close the Selenium webdriver

        return urls

    # Function to scrape data (title and story) from each topic URL
    def scrape_topic_data(self, urls):
        data = []
        for u in urls:
            try:
                req = requests.get(u)
                bs = BeautifulSoup(req.content, 'html.parser')

                # Find title and story elements and extract text
                for k in bs.find_all('div', {'class': 'display-post-status-leftside'}):
                    topic_element = k.find('h2', {'class': 'display-post-title'})
                    if topic_element:
                        topic = topic_element.text.strip().replace('\n', '').replace('\t', '').replace('\xa0', '').replace('*', '').replace('\u200b', '')

                        story = bs.find('div', {'class': 'display-post-story'})
                        if story:
                            story_text = story.text.strip().replace('\n', '').replace('\t', '').replace('\xa0', '').replace('*', '').replace('\u200b', '')

                            # Append the data as a dictionary to the list
                            data.append({'url': u, 'title': topic, 'story': story_text})
            except Exception as e:
                print(f"An error occurred while scraping data from URL '{u}': {e}")

        return data

    # Function to convert scraped data to DataFrame
    def to_dataframe(self, data):
        return pd.DataFrame(data)

Writing PantipTagScraper.py


In [56]:
# Example usage
scraper = PantipScraper(scroll_iterations=0)  # Initialize the scraper with 2 scroll iterations
urls = scraper.get_topic_urls(['ประกันชีวิต'])  # Get topic URLs for specified tag
data = scraper.scrape_topic_data(urls[0:10])  # Scrape data (title and story) from each URL
df = scraper.to_dataframe(data)  # Convert scraped data to DataFrame

In [58]:
df

,url,title,story
0,https://pantip.com/topic/42470306,ประกันสุขภาพที่ไหนดีคะ 2567,คือเราอยากทำประกันสุขภาพ+อุบัติเหตุ ที่ไม่ต้อง...
1,https://pantip.com/topic/41850025,ประกันสุขภาพแบบไหนดี 2566,ช่วยแนะนำทีค่ะ อยากได้ประกันสุขภาพที่ครอบคลุมท...
2,https://pantip.com/topic/42660160,ยกเลิกประกันชีวิตดีไหมครับ,สมัยทำงานแรกๆเคยซื้อประกันชีวิตไว้ให้พ่อเป็นแบ...
3,https://pantip.com/topic/42369379,ตกงาน!! 6เดือนแล้ว มีแต่คนชวนไปขายประกัน ควรลอ...,กำลังเครียดมากเลยค่ะว่างงานมา6เดือนแล้วสัมภาษณ...
4,https://pantip.com/topic/41641266,ต้องการยกเลิกกรมธรรม์ FWD สะสมทรัพย์ 10/20,รบกวนสอบถามครับพอดีมีเจ้าหน้าที่ FWD เสนอโปรโม...
5,https://pantip.com/topic/42659926,กรุงไทยAxa ไม่เคลมมีใบประกาศ,เรื่องมีอยู่ว่าลูกสาวนอนรพล่าสุดเป็นไข้หวัดใหญ...
6,https://pantip.com/topic/42479831,คิดจะทำประกันชีวิตตอนอายุ 35 ช้าไปมั้ยครับ,คิดจะทำประกันชีวิตตอนอายุ35ช้าไปมั้ยครับเห็นเพ...
7,https://pantip.com/topic/42228144,ทำไมคนต้องการยกเลิก ประกัน อุบัติเหตุ ของ SCB ...,คือพอดีพึ่งมีสายจากตัวแทนขายประกันโทรมาเมื่อตอ...
8,https://pantip.com/topic/42545425,อยากซื้อประกันสุขภาพค่ะ 2567,อายุจะ23อีกประมาณ2อาทิตย์นี้ค่ะ อยากหาประกันสุ...
9,https://pantip.com/topic/42058854,การดำเนินการเวนคืนกรมธรรม์ FWD ดำเนินการอย่างไ...,พอดีทำประกันกับ FWD ไว้และต้องการยกเลิกเวนคืนก...
